In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
path = '/mnt/ssd1/mehul_data/research2/ABIDE_fc_data.npy'
data = np.load(path, allow_pickle=True).item()

In [4]:
data.keys()


dict_keys(['corr', 'label', 'site', 'age', 'sex'])

In [5]:
# type(data['corr'])
lengths_dict = dict()
for sub in data['corr']:
    # lengths_set.add(len(sub[0]))
    lengths_dict[sub[0].shape[0]] = lengths_dict.get(sub[0].shape[0], 0) + 1
print(lengths_dict)

{116: 119, 196: 128, 236: 85, 78: 25, 176: 210, 296: 119, 246: 56, 146: 59, 152: 28, 206: 28, 124: 4, 316: 3, 232: 1, 202: 1}


## Model

In [12]:
class stDNN(nn.Module):
    def __init__(self, num_channels=246, num_classes=2):
        super(stDNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=256, kernel_size=7)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=7)
        self.conv3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=4, stride=2)
        self.fc = nn.Linear(512, 1)
        
    def forward(self, x):
        # First 1D CNN block
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # Second 1D CNN block
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        # Temporal averaging
        x = torch.mean(x, 2)
        # Classification layer
        x = torch.sigmoid(self.fc(x))
        return x


## Synthetic Dataset and DataLoader

In [13]:
from torch.utils.data import Dataset, DataLoader
import torch

class fMRIDataset(Dataset):
    def __init__(self, x, y):
        """
        Args:
            x (Tensor): Input features with shape (num_samples, num_channels, sequence_length).
            y (Tensor): Labels with shape (num_samples,).
        """
        assert x.size(0) == y.size(0), "The number of samples in x and y should be equal."
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.size(0)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the sample to return.
        Returns:
            tuple: (feature, label) where feature is the input data at index `idx` and label is its corresponding label.
        """
        return self.x[idx], self.y[idx]

# Create synthetic data
num_samples = 1000
num_channels = 246
num_classes = 2
sequence_length = 1000  # Assuming each sample is a time series with 1000 time points

# Synthetic features
x = torch.randn(num_samples, num_channels, sequence_length)
# Synthetic labels: Generate random binary labels
y = torch.randint(0, num_classes, (num_samples,))

# Instantiate the dataset
synthetic_dataset = fMRIDataset(x, y)

# Create a DataLoader
batch_size = 32
dataloader = DataLoader(synthetic_dataset, batch_size=batch_size, shuffle=True)

# Example of iterating over the DataLoader
for batch_idx, (features, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}")
    print(f"Feature batch shape: {features.size()}")
    print(f"Labels batch shape: {labels.size()}")
    # Here, you can feed 'features' and 'labels' to your model for training
    break  # Breaking here just to demonstrate; remove this in real training loops

Batch 1
Feature batch shape: torch.Size([32, 246, 1000])
Labels batch shape: torch.Size([32])


In [14]:
from torch.utils.data import random_split

def create_data_loaders(dataset, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, batch_size=32):
    assert train_ratio + val_ratio + test_ratio == 1, "Ratios must sum to 1"
    
    total_size = len(dataset)
    train_size = int(total_size * train_ratio)
    val_size = int(total_size * val_ratio)
    test_size = total_size - train_size - val_size  # Ensures all samples are used
    
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # Shuffle=False for test set
    
    return train_loader, val_loader, test_loader

# Using the function to create DataLoader instances for train, val, and test
train_loader, val_loader, test_loader = create_data_loaders(synthetic_dataset, batch_size=batch_size)


In [15]:
train_loader

## Training Class

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score


class stDNNTrainer:
    def __init__(self, model, train_loader, val_loader=None, batch_size=32, learning_rate=0.0001, save_path='best_model.pth'):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        self.best_val_loss = float('inf')
        self.save_path = save_path  # Path to save the best model
    
    def train_epoch(self):
        self.model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        all_probs = []
        
        for inputs, labels in self.train_loader:
            inputs, labels = inputs.float(), labels.float()
            self.optimizer.zero_grad()
            
            outputs = self.model(inputs)
            loss = self.criterion(outputs.squeeze(), labels)
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            probs = torch.sigmoid(outputs.squeeze())
            preds = torch.round(probs)
            print(preds)
            print(labels)
            break
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
        
        avg_loss = running_loss / len(self.train_loader)
        accuracy, precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro', zero_division=0)
        try:
            auc = roc_auc_score(all_labels, all_probs)
        except ValueError:
            auc = np.nan  # Handle cases where AUC can't be computed
        
        return avg_loss, accuracy, precision, recall, f1, auc
    
    def train(self, epochs):
        for epoch in range(epochs):
            train_loss, accuracy, precision, recall, f1, auc = self.train_epoch()
            print(f"Epoch {epoch+1}/{epochs}, Train - Loss: {train_loss:.4f}, Acc: {accuracy:.4f}, Prec: {precision:.4f}, Rec: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")
            
            if self.val_loader:
                val_metrics = self.validate()
                print(f"Validation - Loss: {val_metrics['val_loss']:.4f}, Acc: {val_metrics['accuracy']:.4f}, Prec: {val_metrics['precision']:.4f}, Rec: {val_metrics['recall']:.4f}, F1: {val_metrics['f1']:.4f}, AUC: {val_metrics['auc']:.4f}")
                print()
                # Save the best model based on validation loss
                if val_metrics['val_loss'] < self.best_val_loss:
                    self.best_val_loss = val_metrics['val_loss']
                    torch.save(self.model.state_dict(), self.save_path)
                    print(f"Saved new best model at epoch {epoch+1}")

    
    def validate(self):
        self.model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []
        all_probs = []
        
        with torch.no_grad():
            for inputs, labels in self.val_loader:
                inputs, labels = inputs.float(), labels.float()
                outputs = self.model(inputs)
                loss = self.criterion(outputs.squeeze(), labels)
                val_loss += loss.item()
                
                probs = torch.sigmoid(outputs.squeeze())
                preds = torch.round(probs)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
        
        avg_val_loss = val_loss / len(self.val_loader)
        
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        
        # Handle AUC calculation safely
        try:
            auc = roc_auc_score(all_labels, all_probs)
        except ValueError:
            auc = np.nan  # Assign NaN if AUC can't be computed
        
        metrics = {
            "val_loss": avg_val_loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "auc": auc
        }
        
        return metrics


## Complete Pipeline Integration

In [17]:
from sklearn.model_selection import KFold

def cross_validate_model(model_class, dataset, num_splits=5, batch_size=32, epochs=15, learning_rate=0.0001):
    kf = KFold(n_splits=num_splits, shuffle=True)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
        print(f"Fold {fold + 1}/{num_splits}")

        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_subsampler)
        
        model = model_class(num_channels=246, num_classes=1)  # Ensure this matches your model's expected input
        trainer = stDNNTrainer(model, train_loader, val_loader, batch_size, learning_rate)
        
        trainer.train(epochs)
        metrics = trainer.validate()  # Collect metrics from validation
        
        print(f"Fold {fold+1} Metrics: {metrics}")  # Optionally print out fold metrics here for immediate feedback
        
        fold_results.append(metrics)
        
    return fold_results


In [18]:
# Create synthetic data
num_samples = 1000
num_channels = 246
num_classes = 2
sequence_length = 1000  # Assuming each sample is a time series with 1000 time points

# Synthetic features
x = torch.randn(num_samples, num_channels, sequence_length)
# Synthetic labels: Generate random binary labels
y = torch.randint(0, num_classes, (num_samples,))

# Instantiate the dataset
synthetic_dataset = fMRIDataset(x, y)

cross_validate_model(stDNN, synthetic_dataset, num_splits=2, batch_size=32, epochs=15, learning_rate=0.0001)


Fold 1/2
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<RoundBackward0>)
tensor([1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0.,
        0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.])


/home/mehul/.conda/envs/pyg/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/mehul/.conda/envs/pyg/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mehul/.conda/envs/pyg/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: not enough values to unpack (expected 5, got 4)